In [ ]:
# Import libraries
import json
from openai import OpenAI
import gradio as gr
import sqlite3

In [ ]:
# Initialise Ollama
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='hohoho')
MODEL = "gpt-oss:latest"

In [ ]:
# We want a helpful and funny assistant - got to keep it FUN!!!
system_message = """
You are a helpful assistant with a good sense of humour for an Airline called FlightAI.
Give short, funny answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

In [ ]:
# JSON for set_price_function:
set_price_function = {
    "name": "set_ticket_price",
    "description": "Set the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the user wants to set the price for",
            },
            "set_price": {
                "type": "integer",
                "description": "The price to set for a return ticket to the city",
            },
        },
        "required": ["destination_city", "set_price"],
        "additionalProperties": False
    }
}

In [ ]:
# JSON for get_price_function:
get_price_function = {
    "name": "get_ticket_price",
    "description": "Get tihe price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the user wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
# And these are included in a list of tools which LLM can call:
tools = [
    {"type": "function", "function": set_price_function},
    {"type": "function", "function": get_price_function},
]

In [ ]:
# Chat function with while loop and cap the while loop to 10
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = ollama.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    i=0
    while response.choices[0].finish_reason=="tool_calls" and i<10:
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = ollama.chat.completions.create(model=MODEL, messages=messages, tools=tools)
        i += 1    
    return response.choices[0].message.content

In [ ]:
# Check if a function name is valid and can be called
def call_function_by_name(func_name, *args, **kwargs):
    if not func_name.isidentifier():
        raise ValueError(f"'{func_name}' is not a valid identifier")
    func = getattr(__import__(__name__), func_name, None)
    if not callable(func):
        raise ValueError(f"No callable function named '{func_name}'")
    return func(*args, **kwargs)

In [ ]:
# Handle tool calls from LLM
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        if arguments.get('set_price'):
            price = arguments.get('set_price')
            price_details = call_function_by_name(tool_call.function.name, city, price)
            responses.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })
        else:
            price_details = call_function_by_name(tool_call.function.name, city) 
            responses.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })
    return responses

In [ ]:
# Create a blank database
DB = "prices.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS prices (city TEXT PRIMARY KEY, price REAL)')
    conn.commit()

In [ ]:
# Same set_ticket_price function as Ed used
def set_ticket_price(city, price):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO prices (city, price) VALUES (?, ?) ON CONFLICT(city) DO UPDATE SET price = ?', (city.lower(), price, price))
        conn.commit()
        return f"Ticket price to {city.title()} has been set to ${price}"

In [ ]:
# Same get_ticket_price function as Ed used
def get_ticket_price(city):
    print(f"DATABASE TOOL CALLED: Getting price for {city}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE city = ?', (city.lower(),))
        result = cursor.fetchone()
        return f"Ticket price to {city.title()} is ${result[0]}" if result else "No price data available for this city"

In [ ]:
# Let's go - starting with a blank database, BUT you can upload multiple prices in a single message
gr.ChatInterface(fn=chat, type="messages").launch()